### 載入依賴

In [ ]:
from configparser import ConfigParser
import pandas as pd
import numpy as np

from Models.DeltaBetaFilter import DeltaBetaFilter
from Models.CutpointCalculator import CutpointCalculator
from Models.GeneFilter import GeneFilter
from Models.AucCalculator import AucCalculator
from Models.GosemsimCalculator import GosemsimCalculator
from Models.CancerStageCalculator import CancerStageCalculator

from MakeFile.FileSaver import FileSaver
from ValidateModel.ValidateData import ValidateData

### 載入設定檔

In [ ]:
_configPath = "Analysis/config.ini"
_config = ConfigParser()
_config.read(_configPath)

### 參數

In [ ]:
# 載入資料
NORMAL_NUM = 50
VALIDATE_NORMAL_NUM = 20
_hyperThreshold = 0.368
_hypoThreshold = -0.238
_pValueThreshold = -np.log10(0.05)

### 載入資料

In [ ]:
# 原始beta資料
_betaDataDf = pd.read_csv(_config["Data"]["BETA_DATA_PATH"])
_betaDataDf.columns.values[0] = "CpG"
# 預篩選DMP資料
_dmpDataDf = pd.read_csv(_config["Data"]["DMP_DATA_PATH"]) 
_dmpDataDf.columns.values[0] = "CpG"
# 標註CpG行
_betaValidateDataDf = pd.read_csv(_config["Data"]["VALIDATE_DATA_PATH"])
_betaValidateDataDf.columns.values[0] = "CpG"

# gene相似度矩陣
_gosemsimDf = pd.read_csv(_config["Data"]["GOSEMSIM_MF_HYPER_PATH"], index_col = 0)

# 共病基因資料
with open(_config["Data"]["GROUP_COMORBIDITY_PATH"], 'r') as file:
    _lines = file.readlines()
_groupComorbidityList = [line.strip() for line in _lines]

### Dbate Analyze

In [ ]:
_deltaBetaFilter = DeltaBetaFilter()
# 計算dbeta
_dbetaDf = _deltaBetaFilter.CalculateDeltaBeta(_betaDataDf, NORMAL_NUM)
# 篩選dbeta
_dbetaDf = _deltaBetaFilter.FilterDeltaBeta(_dbetaDf, _dmpDataDf, onlyPromoter = True)
# 分hyper,hypo
_hyperDf, _hypoDf = _deltaBetaFilter.DetermineDNAm(_dmpDataDf, _hyperThreshold, _hypoThreshold, _pValueThreshold)

### Calculate Cutpoint

In [ ]:

_validateData = ValidateData()
_cutpointCalculator = CutpointCalculator()
# 計算切點
_cutpointDf = _cutpointCalculator.CalculateCutpoint(_betaDataDf, [_hyperDf, _hypoDf], NORMAL_NUM, ["hyper", "hypo"], "CpG", "mid")
# 驗證切點
_cutpointDf = _validateData.ValidateCutpoint(_cutpointDf, _betaValidateDataDf, VALIDATE_NORMAL_NUM)

### Comorbidity Gene Filter

In [ ]:
_geneFilter = GeneFilter()
# 與共病基因交集
_groupDf = _geneFilter.IntersectData(_cutpointDf, _groupComorbidityList, "gene")

### Calculate Auc

In [ ]:
_aucCalculator = AucCalculator()
# 計算AUC
_aucDf = _aucCalculator.CalculateAuc(_betaDataDf, _groupDf, NORMAL_NUM)

In [ ]:

# 
# 計算基因相似度
# 使用R語言gosemsim計算
# 

In [ ]:

_gosemsimCalculator = GosemsimCalculator()
# 分群
_geneCluster, silhouetteFig = _gosemsimCalculator.ClusterGoterm(_gosemsimDf, "ward", range(5, 51))
# 合併資料
_geneCluster = _geneFilter.IntersectData(_aucDf, _geneCluster, "gene")

In [ ]:

# 挑選資料
idx_max_dbeta = _geneCluster.groupby('cluster')['dbeta'].idxmax()
idx_max_auc = _geneCluster.groupby('cluster')['auc'].idxmax()
idx_max_F1 = _geneCluster.groupby('cluster')['F1'].idxmax()

print("Dbeta Max:")
print(_geneCluster.loc[idx_max_dbeta])
print("AUC Max:")
print(_geneCluster.loc[idx_max_auc])
print("F1 Max:")
print(_geneCluster.loc[idx_max_F1])